## Determining the optimal no of hidden layers and neurons for an Artificial Neural Network (ANN)

There are some guidelines and methods that helps in making an informed decision

1. Start simple- Begin with simple architechture and gradually increase complexity if needed 

2. Grid Search/Random Search - use grid search/random search for trying different architechture

3. Cross-validation -use cross-validation for evaluation of the different architecture 


- Heuristics and rules of thumb:
1. The number of neuron in the hidden layer should be between the size of the input and the output layers.

2. A common practice is to start with 1-2 hidden layers

In [2]:
## Experiment 

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler,LabelEncoder,OneHotEncoder

from sklearn.pipeline import Pipeline
from scikeras.wrappers import KerasClassifier

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense 
from tensorflow.keras.callbacks import EarlyStopping
import pickle


In [4]:
data=pd.read_csv('D:\Genai-lc-hf\Datasets\Churn_Modelling.csv')

In [5]:
label_encoder_gender=LabelEncoder()
data['Gender']=label_encoder_gender.fit_transform(data['Gender'])

onehot_encoder_geo=OneHotEncoder(handle_unknown='ignore')
onehot_encoded=onehot_encoder_geo.fit_transform(data[['Geography']]).toarray()

geo_encoded_df=pd.DataFrame(onehot_encoded,columns=onehot_encoder_geo.get_feature_names_out(['Geography']))

data=pd.concat([data.drop('Geography',axis=1),geo_encoded_df],axis=1)

In [6]:
X = data.drop(['RowNumber', 'CustomerId', 'Surname', 'Exited'], axis=1)
y = data['Exited']

In [7]:


X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [8]:
with open('label_encoder_gender.pkl','wb') as f:
    pickle.dump(label_encoder_gender,f)
with open('onehot_encoder_geo.pkl','wb') as f:
    pickle.dump(onehot_encoder_geo,f)
with open('scaler.pkl','wb') as f:
    pickle.dump(scaler,f)


In [9]:
# defining a function to create the model and try different parameters(KerasClassifier)

def create_model(neurons=32,layers=1):
    model=Sequential()
    model.add(Dense(neurons,activation='relu',input_shape=(X_train.shape[1],))) #by default input layer is added
    
    for _ in range (layers-1):
        model.add(Dense(neurons,activation='relu'))
    model.add(Dense(1,activation='sigmoid')) #by default output layer is added 
    model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
    
    return model

In [10]:
#creating a keras classifier 

model=KerasClassifier(layers=1,neurons=32,build_fn=create_model,epochs=50,batch_size=10,verbose=0)

In [11]:
#defining the grid search params 

param_grid={
    'neurons':[16,32,64,128],
    'layers':[1,2,3],
    'epochs':[50,100]
}

In [ ]:
grid=GridSearchCV(estimator=model,param_grid=param_grid,n_jobs=1,cv=3)

grid_result=grid.fit(X_train,y_train)

print("Best: %f using %s"%(grid_result.best_score_,grid_result.best_params_))

d:\Genai-lc-hf\tfenv\lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
d:\Genai-lc-hf\tfenv\lib\site-packages\keras\src\layers\core\dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
d:\Genai-lc-hf\tfenv\lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
d:\Genai-lc-hf\tfenv\lib\site-packages\keras\src\layers\core\dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `